[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2024/blob/main/40--spark/01--rdd.ipynb)

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz -O spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz

In [1]:
!pip install -q pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("DDD")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

23/01/20 20:55:33 WARN Utils: Your hostname, Jakubs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.5 instead (on interface en0)
23/01/20 20:55:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/20 20:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
sc = spark.sparkContext

### Resilient Distributed Dataset or RDD

An RDD is a distributed collection of elements. All work in Spark is expressed as either creating new RDDs, transforming existing RDDs, or calling actions on RDDs to compute a result. Spark automatically distributes the data contained in RDDs across your cluster and parallelizes the operations you perform on them.

https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds

#### Creating and RDD using parallelize
Another way of creating an RDD is to parallelize an already existing list.

In [4]:
A = ((a, a*a) for a in range(1000))

In [6]:
data = sc.parallelize(A)

In [7]:
data.count()

1000

In [8]:
data.take(10)

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [9]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt -O var/krzyzacy-1.txt

--2023-01-20 20:57:05--  https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 740368 (723K) [text/plain]
Saving to: ‘var/krzyzacy-1.txt’

var/krzyzacy-1.txt  100%[===================>] 723,02K  --.-KB/s    in 0,08s   

2023-01-20 20:57:05 (8,31 MB/s) - ‘var/krzyzacy-1.txt’ saved [740368/740368]



In [10]:
file = sc.textFile('var/krzyzacy-1.txt')

In [12]:
kDF = spark.read.text('var/krzyzacy-1.txt')

In [28]:

X = spark.createDataFrame([("Krakow", "1", {"foo": "boo"}), ("Warszawa", "2", {})], ['City', "digit", "attr"])

In [29]:
X.show()

+--------+-----+------------+
|    City|digit|        attr|
+--------+-----+------------+
|  Krakow|    1|{foo -> boo}|
|Warszawa|    2|          {}|
+--------+-----+------------+



In [46]:
!echo '{"city":"Lublin","digit":5,"attr":{"foo":"zoo"}}' > cities.list
!echo '{"city":"Bielski","digit":2,"attr":{"sigma":"gamma"}}' >> cities.list

In [47]:
cat cities.list

{"city":"Lublin","digit":5,"attr":{"foo":"zoo"}}
{"city":"Bielski","digit":2,"attr":{"sigma":"gamma"}}


In [57]:
c = (spark.read
     .option('dropFieldIfAllNull', True)
     .option("primitivesAsString", True)
     .json("cities.list")
    )

In [58]:
c.printSchema()

root
 |-- attr: struct (nullable = true)
 |    |-- foo: string (nullable = true)
 |    |-- sigma: string (nullable = true)
 |-- city: string (nullable = true)
 |-- digit: string (nullable = true)



In [60]:
import pyspark.sql.functions as F

In [64]:
multiplicate = F.udf(lambda x: int(x)*2)

In [69]:
c = c.withColumn("multipled", multiplicate(F.col("digit")))
c = c.withColumnRenamed('digit', 'number')

In [77]:
c.select(['attr.foo','city']).where('(attr.foo is not null)').show()

+---+------+
|foo|  city|
+---+------+
|zoo|Lublin|
+---+------+



In [55]:
import pyspark.sql.functions as F

In [78]:
T = spark.createDataFrame([
    ('nice one', ['foo', 'moo', 'boo', 'zoo']),
    ('bad one', ['foo', 'moo',])
],
['title', 'tags']
)

In [100]:
T.show()

+--------+--------------------+
|   title|                tags|
+--------+--------------------+
|nice one|[foo, moo, boo, zoo]|
| bad one|          [foo, moo]|
+--------+--------------------+



In [101]:
T.printSchema()

root
 |-- title: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [106]:
T.select('title', F.explode('tags')).show()

+--------+---+
|   title|col|
+--------+---+
|nice one|foo|
|nice one|moo|
|nice one|boo|
|nice one|zoo|
| bad one|foo|
| bad one|moo|
+--------+---+



In [107]:
T.select('title', F.explode_outer('tags')).show()

+--------+---+
|   title|col|
+--------+---+
|nice one|foo|
|nice one|moo|
|nice one|boo|
|nice one|zoo|
| bad one|foo|
| bad one|moo|
+--------+---+



In [115]:
spark.sql("select explode(sequence(1,1000))").show()

+---+
|col|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
+---+
only showing top 20 rows



In [132]:
spark.sql('''
select dayofweek(now()), next_day(now(), "Mon")
''').show()

+----------------+--------------------+
|dayofweek(now())|next_day(now(), Mon)|
+----------------+--------------------+
|               6|          2023-01-23|
+----------------+--------------------+



In [102]:
T.select(F.spark_partition_id(), F.expr('version()')).show()

+--------------------+--------------------+
|SPARK_PARTITION_ID()|           version()|
+--------------------+--------------------+
|                   0|3.3.1 fbbcf9434ac...|
|                   0|3.3.1 fbbcf9434ac...|
+--------------------+--------------------+



In [84]:
k = kDF.withColumn("NewCol", F.col("value"))

In [85]:
k = k.drop("value")

In [86]:
k.show()

+--------------------+
|              NewCol|
+--------------------+
|  Henryk Sienkiewicz|
|                    |
|            Krzyżacy|
|               Tom I|
|                    |
|ISBN 978-83-288-2...|
|                    |
|                    |
|                    |
|                    |
|   Rozdział pierwszy|
|                    |
|W Tyńcu, w gospod...|
|                    |
|Tuż przy nim za s...|
|                    |
|Gospodarz Niemiec...|
|                    |
|Jeszcze ciekawiej...|
|                    |
+--------------------+
only showing top 20 rows



In [98]:
k = k.withColumns({'starts_with1': F.col("NewCol"), 'x': F.col("NewCol"), 'y': F.col('NewCol').startswith('A')})

In [99]:
k.filter(k['y'] == True).show()

+--------------------+--------------------+--------------------+----+
|              NewCol|        starts_with1|                   x|   y|
+--------------------+--------------------+--------------------+----+
|  A Zbyszko odrzekł:|  A Zbyszko odrzekł:|  A Zbyszko odrzekł:|true|
|A wtem przez drzw...|A wtem przez drzw...|A wtem przez drzw...|true|
|A Zbyszkowi zaświ...|A Zbyszkowi zaświ...|A Zbyszkowi zaświ...|true|
|A przetowłosa Dan...|A przetowłosa Dan...|A przetowłosa Dan...|true|
|  A Zbyszko odrzekł:|  A Zbyszko odrzekł:|  A Zbyszko odrzekł:|true|
|  A potem do Danusi:|  A potem do Danusi:|  A potem do Danusi:|true|
|A zakonnicy nie o...|A zakonnicy nie o...|A zakonnicy nie o...|true|
|A Maćko śmiał się...|A Maćko śmiał się...|A Maćko śmiał się...|true|
|A potem, wyciągną...|A potem, wyciągną...|A potem, wyciągną...|true|
|Ale jej nie zbudz...|Ale jej nie zbudz...|Ale jej nie zbudz...|true|
|A oni rzeczywiści...|A oni rzeczywiści...|A oni rzeczywiści...|true|
|A Zbyszkowi rozja..

In [11]:
file

var/krzyzacy-1.txt MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [25]:
file.count()

7607

In [26]:
file.take(10)

['Henryk Sienkiewicz',
 '',
 'Krzyżacy',
 'Tom I',
 '',
 'ISBN 978-83-288-2813-1',
 '',
 '',
 '',
 '']

In [27]:
filtered = file.filter(lambda x: x != '')

In [28]:
filtered.take(10)

['Henryk Sienkiewicz',
 'Krzyżacy',
 'Tom I',
 'ISBN 978-83-288-2813-1',
 'Rozdział pierwszy',
 'W Tyńcu, w gospodzie „Pod Lutym Turem”, należącej do opactwa, siedziało kilku ludzi, słuchając opowiadania wojaka bywalca, który z dalekich stron przybywszy, prawił im o przygodach, jakich na wojnie i w czasie podróży doznał. Człek był brodaty, w sile wieku, pleczysty, prawie ogromny, ale wychudły; włosy nosił ujęte w pątlik, czyli w siatkę naszywaną paciorkami; na sobie miał skórzany kubrak z pręgami wyciśniętymi przez pancerz, na nim pas, cały z miedzianych klamr; za pasem nóż w rogowej pochwie, przy boku zaś krótki kord podróżny.',
 'Tuż przy nim za stołem siedział młodzieńczyk o długich włosach i wesołym spojrzeniu, widocznie jego towarzysz lub może giermek, bo przybrany także po podróżnemu, w taki sam powyciskany od zbroicy skórzany kubrak. Resztę towarzystwa stanowiło dwóch ziemian z okolic Krakowa i trzech mieszczan w czerwonych składanych czapkach, których cienkie końce zwieszały si

In [29]:
counts = filtered \
    .flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda acc, item: acc + item)


In [30]:
counts.takeOrdered(5)

[('', 24),
 ('(bo', 1),
 ('(http://creativecommons.org/licenses/by-sa/3.0/).', 1),
 ('(http://wolnelektury.pl).', 1),
 ('(precz', 1)]

In [31]:
counts.takeOrdered(5, key=lambda x: -x[1])

[('—', 4339), ('i', 3906), ('się', 2486), ('nie', 2024), ('w', 2017)]

In [32]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-drugi.txt -O var/krzyzacy-2.txt

--2021-12-11 07:13:17--  https://wolnelektury.pl/media/book/txt/krzyzacy-tom-drugi.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 840889 (821K) [text/plain]
Saving to: ‘var/krzyzacy-2.txt’

var/krzyzacy-2.txt  100%[===================>] 821.18K  --.-KB/s    in 0.1s    

2021-12-11 07:13:17 (8.38 MB/s) - ‘var/krzyzacy-2.txt’ saved [840889/840889]



In [33]:
file = sc.textFile('var/krzyzacy*')

In [34]:
file.count()

15477

In [147]:
A = spark.createDataFrame([(a,) for a in range(1,100,1)], ['col'])

In [171]:
A.sample(0.05, seed=17).show()

+---+
|col|
+---+
|  9|
| 35|
| 72|
+---+



In [ ]:
A.

In [ ]:
spark.stop()